#### Import Julia packages

In [371]:
import Pkg
# Pkg.add("JSON3")
#Pkg.add("Interpolations")

In [372]:
using Gridap
using GridapMakie, CairoMakie, FileIO
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.Arrays
using Gridap.Algebra
using Gridap.Geometry
using Gridap.Fields
using Gridap.CellData
using FillArrays
using Test
using InteractiveUtils
using PyPlot
using JSON3
using Dates
using Interpolations

In [373]:
function mark_nodes(f,model::DiscreteModel)
  topo   = get_grid_topology(model)
  coords = get_vertex_coordinates(topo)
  mask = map(f,coords)
  return mask
end
function update_labels!(e::Integer,model::CartesianDiscreteModel,f_Γ::Function,name::String)
    mask = mark_nodes(f_Γ,model)
    _update_labels_locally!(e,model,mask,name)
    nothing
end
function _update_labels_locally!(e,model::CartesianDiscreteModel{2},mask,name)
  topo   = get_grid_topology(model)
  labels = get_face_labeling(model)
  cell_to_entity = labels.d_to_dface_to_entity[end]
  entity = maximum(cell_to_entity) + e
  # Vertices
  vtxs_Γ = findall(mask)
  vtx_edge_connectivity = Array(get_faces(topo,0,1)[vtxs_Γ])
  # Edges
  edge_entries = [findall(x->any(x .∈  vtx_edge_connectivity[1:end.!=j]),
    vtx_edge_connectivity[j]) for j = 1:length(vtx_edge_connectivity)]
  edge_Γ = unique(reduce(vcat,getindex.(vtx_edge_connectivity,edge_entries),init=[]))
  labels.d_to_dface_to_entity[1][vtxs_Γ] .= entity
  labels.d_to_dface_to_entity[2][edge_Γ] .= entity
  add_tag!(labels,name,[entity])
  return cell_to_entity
end

_update_labels_locally! (generic function with 1 method)

#### Import Custom functions

In [374]:
#include("Julia_functions/solution_animation.jl")
include("Julia_functions/indicator_chi.jl")
include("Julia_functions/cost_fun.jl")
include("Julia_functions/gradient_descent.jl")
include("Julia_functions/solvers.jl")
include("Julia_functions/find.jl")
include("Julia_functions/res_plot.jl")
include("Julia_functions/get_domain.jl")
include("Julia_functions/get_price_temp.jl")

get_price_temp (generic function with 1 method)

#### Read JSON File

In [375]:
width, height, windows, doors, heating_elem = get_room("Boundry.json")

(5, 3, Any[(0.3, 1.3, "Top"), (3.3, 4.3, "Top")], Any[(0.4, 1.3, "Right")], [[0.2, 2.0, 0.1, 0.3]])

In [376]:
price_EUR, price_NOK, T_out, hour_lst = get_price_temp("WeatherandEnergy.json")

(Any[0.01669, 0.01319, 0.00967, 0.01037, 0.02006, 0.02314, 0.01764, 0.01828, 0.01869, 0.01863, 0.01817, 0.01777], Any[0.19048, 0.15054, 0.11036, 0.11835, 0.22894, 0.2641, 0.20133, 0.20863, 0.21331, 0.21262, 0.20737, 0.20281], Any[282.15, 282.34999999999997, 282.25, 282.34999999999997, 282.15, 281.45, 280.65, 280.84999999999997, 280.75, 280.45, 280.25, 279.95], Any[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21])

#### Boundary conditions

In [377]:
function generate_f_Γ(bound, width, height)
    return function (x)
        any(
            (b[3] == "Top"    && (b[1] ≤ x[1] ≤ b[2]) && (x[2] ≈ height)) ||
            (b[3] == "Bottom" && (b[1] ≤ x[1] ≤ b[2]) && (x[2] ≈ 0)) ||
            (b[3] == "Left"   && (b[1] ≤ x[2] ≤ b[2]) && (x[1] ≈ 0)) ||
            (b[3] == "Right"  && (b[1] ≤ x[2] ≤ b[2]) && (x[1] ≈ width))
            for b in bound
        )
    end
end

generate_f_Γ (generic function with 1 method)

In [378]:
f_Γ_w = generate_f_Γ(windows, width, height)
f_Γ_d = generate_f_Γ(doors, width, height)
f_Γ_wall(x) = !(f_Γ_w(x) || f_Γ_d(x))

f_Γ_wall (generic function with 1 method)

#### Domain & Discretization

In [379]:
domain = (0, width , 0, height)
partition = (25,25)
model = CartesianDiscreteModel(domain,partition) 
update_labels!(1, model, f_Γ_w, "window")
update_labels!(2, model, f_Γ_d, "door")
update_labels!(3, model, f_Γ_wall, "wall")

order = 1
degree = 2*order

Ω = Triangulation(model)
dΩ = Measure(Ω, degree)

Γ_w = BoundaryTriangulation(model, tags="window")
dΓ_w = Measure(Γ_w, degree)

Γ_d = BoundaryTriangulation(model, tags="door")
dΓ_d = Measure(Γ_d, degree)

Γ_wall = BoundaryTriangulation(model, tags="wall")
dΓ_wall = Measure(Γ_wall, degree)

reffe = ReferenceFE(lagrangian,Float64,order)
Testspace = TestFESpace(model,reffe,conformity=:H1)                         
Trialspace = TransientTrialFESpace(Testspace)                                
Uspace = FESpace(model, reffe, conformity=:H1)

# fig = CairoMakie.plot(Ω)
# scatter!(Ω, marker=:circle, markersize=20, color=:blue) 
# display(fig)

UnconstrainedFESpace()

In [380]:
function get_price_function(price_NOK, tF,σ=0.001)
    time = LinRange(0, tF, length(price_NOK))  # Lager tidsintervall
    
    return function (t)
        for i in 1:length(time)-1
            if t >= time[i] && t < time[i+1]
                return price_NOK[i]*σ
            end
        end
        return price_NOK[end]*σ  # Returnerer siste verdi hvis t er større enn siste tidsverdi
    end
end

function get_temp_function(temp, tF)
    time = LinRange(0, tF, length(temp))  # Lager tidsintervall
    
    return function (t)
        for i in 1:length(time)-1
            if t >= time[i] && t < time[i+1]
                return temp[i]
            end
        end
        return temp[end]  # Returnerer siste verdi hvis t er større enn siste tidsverdi
    end
end

get_temp_function (generic function with 1 method)

#### Time parameters

In [ ]:
t0 = 0.0    # Start time
tF = 10.0   # End time
Δt = 0.05   # Timestep

5

#### Room parameters

In [ ]:
h_wall(x) = 0.22*1000            # [W/m^2K]
h_window(x) = 1.2*1000
h_door(x) = 1.0*1000

ρ(x)=1.225
c(x)=1020.0
k(x)=0.025*1000

h = (h_wall, h_window, h_door) 
constants = (c, ρ, k, h) 

#Toutdoor(x,t)=273.0
#Tout(t)=x->Toutdoor(x,t)

Tout = get_temp_function(T_out, tF)

price = get_price_function(price_NOK, tF)

Tini(x)=297.0

TIni=interpolate_everywhere(Tini, Uspace(t0))
Tfin=interpolate_everywhere(297.0, Uspace(tF))

SingleFieldFEFunction():
 num_cells: 625
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 18197695399889677620

#### Controll parameter

In [383]:
function heating_elements(heating_elem)
    return function (x) sum(χ(x[1], elem[1], elem[2]) * χ(x[2], elem[3], elem[4]) for elem in heating_elem) end
end

heating_elements (generic function with 1 method)

In [384]:
#q_pos(x) = χ(x[1], 0.21, 1.73) * χ(x[2], 0.13, 0.16)
q_pos = heating_elements(heating_elem)
Q(x,t)=χ(t,0.0,100.0)*3000.0*q_pos(x)*0.0
Qt(t)=x->Q(x,t)

Qt (generic function with 1 method)

#### Solver parameters

In [385]:
L2norm(u)=√((tF-t0)*∑(Δt*∑(∫(uu⋅uu)*dΩ) for (t,uu) in u))
L2skp(u)=(tF-t0)*∑(Δt*∑(∫(uu⋅uu)*dΩ) for (t,uu) in u)

γ = 5000.0

Proj(a,b,z) = min(max(a,z),b)
a=0.0
b=2000.0
Proj(z) = [(t,FEFunction(Uspace,map(x->Proj(a,b,x), get_free_dof_values(zz)))) for (t,zz) in z]

Proj (generic function with 2 methods)

#### Test Gradient DEs

In [ ]:
(Ts,qs,Ws,costs) = GradientDescent(;solveSE=SEsolver, 
solveAE=AEsolver, 
spaces=(Trialspace, Testspace, Uspace), 
dΩ=dΩ, 
dΓ=(dΓ_w,dΓ_d,dΓ_wall), 
Q=Qt, 
J=E, 
∇f=∇e, 
P=Proj, 
s_min=s_min,
sminargs=nothing, 
saveall=false, 
tol=1e-5, 
iter_max=10,
armijoparas=(ρ=1/2, α_0=1.0, α_min=1e-5, σ=1e-4), 
Δt=Δt, 
t0=t0,
tF=tF,
Tout=Tout,
constants=constants,
Tfin=Tfin,
q_pos=q_pos)

T computed
W computed
entered for loop, E=1.0488299004633555e7, k = 1
α = 1.0, new_cost = 1.0488299004633555e7
α = 0.5, new_cost = 1.8504818503374055e8
α = 0.25, new_cost = 1.8504818503374055e8
α = 0.125, new_cost = 1.8504818503374055e8
α = 0.0625, new_cost = 1.8504818503374055e8
α = 0.03125, new_cost = 1.8504818503374055e8
α = 0.015625, new_cost = 1.8504818503374055e8
α = 0.0078125, new_cost = 1.5448349456520194e8
α = 0.00390625, new_cost = 7.932844898088145e7
α = 0.001953125, new_cost = 3.549620459333078e7
α = 0.0009765625, new_cost = 1.7445711201243356e7
α = 0.00048828125, new_cost = 1.1422966745243767e7
entered for loop, E=9.888024181997884e6, k = 2
α = 1.0, new_cost = 9.888024181997884e6
α = 0.5, new_cost = 1.0472420658394158e7

In [ ]:
# Opprett en mappe for midlertidige filer hvis den ikke eksisterer
if !isdir("tmp")
    mkdir("tmp")
end

# Initialtilstanden og resten av tidsløsningene
# uh0 = solution[1][2]  # Første element er (t0, T0), vi henter T0
# uh  = solution[2:end]  # De resterende løsningene


uh = Ts
# Lagre resultater i Paraview-format (VTK)
createpvd("results") do pvd
    # Første løsning (t=0)
    # pvd[0] = createvtk(Ω, "tmp/results_0.vtu", cellfields=["u" => uh0])

    # Lagrer løsninger for hver tidssteg
    for (tn, uhn) in uh
        pvd[tn] = createvtk(Ω, "tmp/results_$tn.vtu", cellfields=["u" => uhn])
    end
end


In [ ]:
function get_temperature_int(Ts, width, height)
	tmp = []
	for temp in Ts
	push!(tmp, ∑(∫(temp[2])*dΩ))
	end
	return tmp / (width * height)
end

In [ ]:
function get_control_int(qs)
    q = []
	for u in qs
	push!(q, ∑(∫(u[2])*dΩ))
	end
	return q
end

In [ ]:
T = get_temperature_int(Ts, width, height)
q = get_control_int(qs)

In [ ]:
result_plot()